In [ ]:
import Pkg
Pkg.activate(".")
Pkg.add("Revise")
import Revise

Pkg.develop(path="../../..")
import Mycelia

pkgs = [
"DataFrames",
"StatsBase",
"StatsPlots",
"uCSV",
"ProgressMeter"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))

identifiers_to_rerun = [
"SRR6399724",
"SRR6399725",
"SRR6399726",
"SRR6399727",
"SRR6399728",
"SRR6399729",
"SRR6399730",
"SRR6399731",
"SRR6399732",
"SRR6399773",
"SRR6399810",
"SRR6399900",
"SRR6399901",
"SRR6399902",
"SRR6399903",
"SRR6399905",
"SRR6399906",
"SRR6399907",
"SRR6399908",
"SRR6399909",
"SRR6399910",
"SRR6399911",
"SRR6399912",
"SRR6399913",
"SRR6399914",
"SRR6399915",
"SRR6399916",
"SRR6399917",
"SRR6399918",
"SRR6399919",
"SRR6399920",
"SRR6399921",
"SRR6399922",
"SRR6399923",
"SRR6399924",
"SRR6399925",
"SRR6399926",
"SRR6399927",
"SRR6399928",
"SRR6399929",
"SRR6399930",
"SRR6399931",
"SRR6399932",
"SRR6399933",
"SRR6399934",
"SRR6399935",
"SRR6399936",
"SRR6399937",
"SRR6399938",
"SRR6399939",
"SRR6399940",
"SRR6399941",
"SRR6399942",
"SRR6399943",
"SRR6399944",
"SRR6399945",
"SRR6399946",
"SRR6399947",
"SRR6399948",
"SRR6399949",
"SRR6399950",
"SRR6399951",
"SRR6399952",
"SRR6399953",
"SRR6399954",
"SRR6399955",
"SRR6399956",
"SRR6399957",
"SRR6399958",
"SRR6399959",
"SRR6399960",
"SRR6399961",
"SRR6399962",
"SRR6399963",
"SRR6399964",
"SRR6399965",
"SRR6399966",
"SRR6399967",
"SRR6399968",
"SRR6399969",
"SRR6399970",
"SRR6399971",
"SRR6399972",
"SRR6399973",
"SRR6399974",
"SRR6399975",
"SRR6399976",
"SRR6399977",
"SRR6399978",
"SRR6399979",
"SRR6399980",
"SRR6399981",
"SRR6399982",
"SRR6399983",
"SRR6399984",
"SRR6399985",
"SRR6399986",
"SRR6399987",
"SRR6399988",
"SRR6399989",
"SRR6399990",
"SRR6399991",
"SRR6399992",
"SRR6399993",
"SRR6399994",
"SRR6399995",
"SRR6399996",
"SRR6399997",
"SRR6399998",
"SRR6399999",
"SRR6400000",
"SRR6400001",
"SRR6400002",
"SRR6400003",
"SRR6400004",
"SRR6400005",
"SRR6400006",
"SRR6400007",
"SRR6400008",
"SRR6400009",
"SRR6400010",
"SRR6400011",
"SRR6400012",
"SRR6400013",
"SRR6400014",
"SRR6400015",
"SRR6400016",
"SRR6400017",
"SRR6400018",
"SRR6400019",
"SRR6400021",
"SRR6400022",
"SRR6400023",
"SRR6400024",
"SRR7365458",
"SRR7365460",
"SRR7365461",
"SRR7365462",
"SRR7365463",
"SRR7365464",
"SRR7365465",
"SRR7365466",
"SRR7365467",
"SRR7365468",
"SRR7365469",
"SRR7365470",
"SRR7365471",
"SRR7365473",
"SRR7365474",
"SRR7365475",
"SRR7365476",
"SRR7365477",
"SRR7365478",
"SRR7365479",
"SRR7365480",
"SRR7365481",
"SRR7365482",
"SRR7365483",
"SRR7365484",
"SRR7365485",
]

SRR_paths = filter(x -> !(basename(x) in identifiers_to_rerun), SRR_paths)

In [ ]:
results_dir = joinpath(data_dir, "results")

In [ ]:
kraken_db = "k2_pluspfp_20221209"

In [ ]:
taxon_levels = Mycelia.list_ranks()

In [ ]:
# for taxon_level in taxon_levels
taxon_level = first(taxon_levels)
rank_table = Mycelia.list_rank(taxon_level)
rank_taxids = rank_table[!, "taxid"]

In [ ]:
cross_sample_taxon_report = joinpath(results_dir, "$(kraken_db).$(taxon_level).tsv")
cross_sample_taxon_figure_png = joinpath(results_dir, "$(kraken_db).$(taxon_level).png")
# cross_sample_taxon_figure_svg = joinpath(results_dir, "$(kraken_db).$(taxon_level).svg")
cross_sample_taxon_figure_skip_unclassified_png = joinpath(results_dir, "$(kraken_db).$(taxon_level).skip_unclassified.png")
cross_sample_taxon_report_table = DataFrames.DataFrame()
if !isfile(cross_sample_taxon_report)
    ProgressMeter.@showprogress for SRR_path in SRR_paths
        SRR = basename(SRR_path)
        kraken_dir = mkpath(joinpath(SRR_path, "kraken"))
        output_file = joinpath(kraken_dir, "$(SRR).$(kraken_db).kraken-output.tsv")
        report_file = joinpath(kraken_dir, "$(SRR).$(kraken_db).kraken-report.tsv")
        report_table = Mycelia.read_kraken_report(report_file)
        taxon_level_report = report_table[map(x -> x in [rank_taxids..., 0], report_table[!, "ncbi_taxonid"]), :]
        taxon_level_report[!, "sample_identifier"] .= SRR
        append!(cross_sample_taxon_report_table, taxon_level_report)

    end
    uCSV.write(cross_sample_taxon_report, cross_sample_taxon_report_table, delim='\t')
else
    cross_sample_taxon_report_table = DataFrames.DataFrame(uCSV.read(cross_sample_taxon_report, delim='\t', header=1))
end

In [ ]:
cross_sample_taxon_report_table

In [ ]:
cross_sample_taxon_report_summary = cross_sample_taxon_report_table[!, 
    DataFrames.Not([
            "percentage_of_fragments_at_or_below_taxon",
            "number_of_fragments_assigned_directly_to_taxon",
            "rank"
        ])]
cross_sample_taxon_report_summary[!, "taxon"] = map(row -> string(row["ncbi_taxonid"]) * "_" * row["scientific_name"], DataFrames.eachrow(cross_sample_taxon_report_summary))
cross_sample_taxon_report_summary = cross_sample_taxon_report_summary[!, DataFrames.Not([
            "ncbi_taxonid",
            "scientific_name"
        ])]

# assert sortedness & uniqueness (should be a no-op)
unique!(DataFrames.sort!(cross_sample_taxon_report_summary, ["sample_identifier", "taxon"]))

In [ ]:
samples = unique(sort(cross_sample_taxon_report_summary[!, "sample_identifier"]))
taxons = unique(sort(cross_sample_taxon_report_summary[!, "taxon"]))
values = zeros(length(samples), length(taxons))
for (column, taxon) in enumerate(taxons)
    is_taxon = cross_sample_taxon_report_summary[!, "taxon"] .== taxon
    for (row, sample) in enumerate(samples)
        is_sample = cross_sample_taxon_report_summary[!, "sample_identifier"] .== sample
        index = findfirst(is_taxon .& is_sample)
        value = cross_sample_taxon_report_summary[index, "number_of_fragments_at_or_below_taxon"]
        values[row, column] = value
    end
end
values

In [ ]:
aspect_ratio = [9, 1]
scale = 500
StatsPlots.groupedbar(
    values,
    title = "read-classification\n$(taxon_level)",
    xticks = (1:length(samples), samples),
    xlims = (0, length(samples)+1),
    xtickfontsize = 6,
    size= aspect_ratio .* scale,
    xrotation=90,
    ylabel = "number of reads",
    labels = hcat(taxons...),
    bottommargin =80StatsPlots.Plots.PlotMeasures.px,
    leftmargin = 100StatsPlots.Plots.PlotMeasures.px,
    leftbuffer = 0,
    leftpadding = 0,
    legend = :outertopright,
    bar_position = :stack,
    bar_width=0.7
)

In [ ]:
normalized_values = values ./ sum(values, dims=2)

In [ ]:
StatsPlots.groupedbar(
    normalized_values,
    title = "read-classification\n$(taxon_level)",
    xticks = (1:length(samples), samples),
    xlims = (0, length(samples)+1),
    xtickfontsize = 6,
    size= aspect_ratio .* scale,
    xrotation=90,
    ylabel = "proportion of reads",
    labels = hcat(taxons...),
    bottommargin =80StatsPlots.Plots.PlotMeasures.px,
    leftmargin = 100StatsPlots.Plots.PlotMeasures.px,
    leftbuffer = 0,
    leftpadding = 0,
    legend = :outertopright,
    bar_position = :stack,
    bar_width=0.7
)

In [ ]:
# for g in DataFrames.groupby(cross_sample_taxon_report_table, "sample_identifier")
#     # @show sum(g[!, "percentage_of_fragments_at_or_below_taxon"])
#     if sum(g[!, "percentage_of_fragments_at_or_below_taxon"]) == 100
#     else
#         identifier = g[1, "sample_identifier"]
#         println("\"$(identifier)\",")
#     end
# end

In [ ]:
[number_of_fragments_at_or_below_taxon	ncbi_taxonid	scientific_name	sample_identifier

In [ ]:

# Mycelia.list_rank("kingdom")
# Mycelia.list_rank("phylum")
# Mycelia.list_rank("class")
# Mycelia.list_rank("order")
# Mycelia.list_rank("family")
# Mycelia.list_rank("genus")
# Mycelia.list_rank("species")

- loop through the above
- make a matrix where each row is a taxon rank and each column is a dataset
- make stacked barplots for each sample
- run PCA and k-means cluster to find meaningful groups
- repeat for mmseqs and blast nt